# TFM: Análisis de Políticas de Sostenibilidad mediante técnicas de Argumentacion Computacional

## Detección de Argumentos con Qwen 2.5 3B

- ollama serve
- ollama run qwen2.5:3b

In [1]:
#%pip install langchain pymupdf openai openpyxl --quiet



In [1]:
from typing import List
from pydantic import BaseModel, Field, ValidationError
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.exceptions import OutputParserException
import requests
import json
import re
from openai import OpenAI
import openai
import httpx
import pandas as pd
import numpy as np
import os
import openpyxl

process_text_path = "..\\Data\\Processed Files (sections)\\"

model_name="qwen2.5:3b"

## Input text processing

In [ ]:
# 1. Define your Pydantic schema for output
class ArgumentResponse(BaseModel):
    arguments: List[str] = Field(..., description="List of arguments extracted directly from the text.")

# 2. Setup output parser
pydantic_parser = PydanticOutputParser(pydantic_object=ArgumentResponse)

# 3. Extend text with first sentence from the next page
def extend_pages_with_next_sentence(pages):
    def get_first_sentence(text):
        match = re.search(r'(.+?\.)', text.strip())
        return match.group(1).strip() if match else ""

    extended_pages = []
    for i, page in enumerate(pages):
        current_text = page["text"]
        if i + 1 < len(pages):
            next_sentence = get_first_sentence(pages[i + 1]["text"])
            current_text += " " + next_sentence
        extended_pages.append({
            "page": page["page"],
            "text": current_text
        })
    return extended_pages

# 4. Build the prompt and call the LLM to extract arguments
def extract_arguments_json(text, topic, model_name) -> ArgumentResponse:
    format_instructions = pydantic_parser.get_format_instructions()

    prompt = PromptTemplate(
        template=(
            "Task: Text Span Identification for Arguments related to Sustainable Development Goal: {topic}\n\n"
            "Role: You are an expert in logical reasoning, sustainability reporting, and argument analysis. "
            "Your job is to identify and extract **verbatim arguments** about {topic} from long-form sustainability texts.\n\n"
            "Instructions:\n"
            "1. Carefully read the entire input text.\n"
            "2. Identify ONLY those sentences or phrases that:\n"
            "   - Clearly support or argue **for or against** the topic \"{topic}\"\n"
            "   - Contain policy recommendations, problem statements, evaluations, or actions related only to {topic} ignore arguments related to other SGDs\n"
            "   - Are exclusively about {topic} (EXCLUDE if they mention or refer to other SDGs or unrelated sustainability topics)\n\n"
            "3. For each relevant argument, extract the **exact span of text** without paraphrasing.\n"
            "4. Ensure each argument:\n"
            "   - Stands as a full statement\n"
            "   - Matches the original text exactly\n"
            "   - Includes only the necessary context\n\n"
            "Output Rules:\n"
            "- Use **only the exact text** from the original\n"
            "- Do **not** add or reword anything\n"
            "- Return only valid JSON\n"
            "- No markdown (```), no extra explanation\n\n"
            "Text:\n\"\"\"\n{text}\n\"\"\"\n\n"
            "Respond ONLY with a JSON object like this:\n\n"
            "{format_instructions}"
        ),
        input_variables=["text", "topic"],
        partial_variables={"format_instructions": format_instructions}
    )

    final_prompt = prompt.format_prompt(text=text, topic=topic).to_string()

    payload = {
        "model": model_name,
        "prompt": final_prompt,
        "temperature": 0,
        "stream": False
    }

    response = requests.post("http://localhost:11434/api/generate", json=payload)
    if response.status_code != 200:
        raise Exception(f"Ollama error: {response.text}")

    raw_output = response.json()["response"]
    print("Model Output:", raw_output)

    try:
        return pydantic_parser.parse(raw_output)
    except OutputParserException as err:
        print("Parse failed:", err)
        return ArgumentResponse(arguments=[])

# 5. Wrapper function for pipeline
def extract_arguments_from_text(text, topic, model_name) -> List[str]:
    result = extract_arguments_json(text, topic, model_name)
    return result.arguments

# 6. Main document-level processor
def process_document(pages, model_name, topic=""):
    extended_pages = extend_pages_with_next_sentence(pages)
    processed = []
    for page in extended_pages:
        print(f"\n--- Processing Page {page['page']} ---")
        #print("Text to analyze:\n", page["text"])
        
        arguments = extract_arguments_from_text(page["text"], topic, model_name)
        
        print("Extracted Arguments:")
        for i, arg in enumerate(arguments, 1):
            print(f"{i}. {arg}")

        processed.append({
            "page": page["page"],
            "text": page["text"],
            "arguments": arguments
        })
    return processed


# 7. File I/O
def save_to_json(processed, output_path):
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(processed, f, indent=2, ensure_ascii=False)

def process_directory(input_dir, output_dir, prefix, model_name, topic="", sgd_number=None):
    os.makedirs(output_dir, exist_ok=True)
    all_results = []

    for filename in os.listdir(input_dir):
        if filename.endswith(".json") and filename.startswith(prefix):
            filepath = os.path.join(input_dir, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                pages = json.load(f)

            section_name = filename.replace(".json", "")
            processed = process_document(pages, model_name, topic)

            for item in processed:
                item["section"] = section_name  # Add section identifier
                all_results.append(item)
                
    return all_results



## SGD 1: Poverty

In [3]:
topic = "SGD 1 (Poverty): End poverty in all its forms everywhere"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "1"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land), even in countries that are largely to blame for the climate and biodiversity crises."
  ]
}
Extracted Arguments:
1. Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land), even in countries that are largely to blame for the climate and biodiversity crises.

--- Processing Page 8 ---
Model Output: {
  "arguments": [
    "investing in statistical capacity, science, and data literacy are important priorities for achieving the SDGs."
  ]
}

Note: There were no explicit arguments directly supporting or opposing SGD 1 (Poverty) from the provided text. T

## SGD 2: Hunger

In [4]:
topic = "SGD 2 (Hunger): End hunger, achieve food security and improved nutrition and promote sustainable agriculture"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "2"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "To achieve the SDGs the world must both alter its current investment patterns and increase the overall volume of investments."
  ]
}
Extracted Arguments:
1. To achieve the SDGs the world must both alter its current investment patterns and increase the overall volume of investments.

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many developing economies."
  ]
}
Extracted Arguments:
1. UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many developing economies.

--- Processing Page 14 ---
Model Output: {
  "arguments": [
    "All UN Member States should adopt long-term sustainable development pathways that provide a stepwise and medium- to long-term approach to guide their sustainable develop

## SGD 3: Health

In [5]:
topic = "SGD 3 (Health): Ensure healthy lives and promote well-being for all at all ages"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "3"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "At their core, the SDGs are an investment agenda: it is critical that UN Member States adopt and implement the SDG Stimulus and support a comprehensive reform of the global financial architecture."
  ]
}
Extracted Arguments:
1. At their core, the SDGs are an investment agenda: it is critical that UN Member States adopt and implement the SDG Stimulus and support a comprehensive reform of the global financial architecture.

--- Processing Page 8 ---
Model Output: {
  "arguments": [
    "Executive Summary\nFurther investment is needed in statistical capacity and data literacy to support long-term pathways for key SDG transformations."
  ]
}
Extracted Arguments:
1. Executive Summary
Further investment is needed in statistical capacity and data literacy to support long-term pathways for key SDG transformations.

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "Most urgently, UN Member States should adopt an SDG

## SGD 4: Education

In [6]:
topic = "SGD 4 (Education): Ensure inclusive and equitable quality education"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "4"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "All countries, poorer and richer alike, should use the half-way momentum to self-critically review and revise their national SDG strategies and commit to strengthening multilateralism."
  ]
}

Note: There are no other arguments directly related to SGD 4 (Education): Ensure inclusive and equitable quality education in the provided text.
Extracted Arguments:
1. All countries, poorer and richer alike, should use the half-way momentum to self-critically review and revise their national SDG strategies and commit to strengthening multilateralism.

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "Most urgently, UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many developing economies."
  ]
}
Extracted Arguments:
1. Most urgently, UN Member States should adopt an SDG Stimulus, to close the

## SGD 5: Gender

In [7]:
topic = "SGD 5 (Gender): Achieve gender equality and empower all women and girls"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "5"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "In LICs and LMICs, national and subnational governments and private businesses need to receive greatly increased funding for SDG investments."
  ]
}
Extracted Arguments:
1. In LICs and LMICs, national and subnational governments and private businesses need to receive greatly increased funding for SDG investments.

--- Processing Page 8 ---
Model Output: {
  "arguments": [
    "The SDSN was created in 2012, soon after the Rio+20 Summit, to mobilize the world’s universities, think tanks, and national laboratories on behalf of the SDGs."
  ]
}

Note: After reviewing the provided text, no arguments directly related to SGD 5 (Gender) were found. The text focuses more on the need for statistical capacity and data literacy rather than gender equality specifically.
Extracted Arguments:
1. The SDSN was created in 2012, soon after the Rio+20 Summit, to mobilize the world’s universities, think tanks, and national laboratories on beh

## SGD 6: Water and sanitation

In [8]:
topic = "SGD 6 (Water and sanitation): Ensure availability and sustainable management of water and sanitation for all"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "6"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)




--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "All countries, poorer and richer alike, should use the half-way momentum to self-critically review and revise their national SDG strategies and commit to strengthening multilateralism."
  ]
}

Note: The provided text does not contain any explicit arguments supporting or refuting SGD 6 (Water and sanitation) directly. Therefore, only a partially relevant argument is identified here, though it pertains to the overall SDGs indirectly.
Extracted Arguments:
1. All countries, poorer and richer alike, should use the half-way momentum to self-critically review and revise their national SDG strategies and commit to strengthening multilateralism.

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "As UN Member States prepare to meet in September to review the progress towards achieving the SDGs, at the midpoint of the 2030 Agenda, SDSN

## SGD 7: Clean Energy

In [9]:
topic = "SGD 7 (Clean Energy): Ensure access to affordable, reliable, sustainable and modern energy for all"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "7"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)"
  ]
}

Note: The JSON instance only includes one argument that directly supports the topic "SGD 7 (Clean Energy): Ensure access to affordable, reliable, sustainable and modern energy for all".
Extracted Arguments:
1. Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "Increased funding from the multilateral development banks (MDBs) and public development banks (PDBs) to low- and middle-income coun

## SGD 8: Decent Work, Economic Growth

In [10]:
topic = "SGD 8 (decent work, economic growth): Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "8"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)"
  ]
}
Extracted Arguments:
1. only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "Most urgently, UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many developing economies."
  ]
}
Extracted Arguments:
1. Most urgently, UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many developing economies.

--- Processing Page 

## SGD 9: Infrastructure, industrilization, innovation

In [11]:
topic = "SGD 9 (Infrastructure, industrilization, innovation): Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "9"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
    "arguments": [
        "Greatly increase funding to national and subnational governments and private businesses, especially in LICs and LMICs, to carry out needed SDG investments."
    ]
}
Extracted Arguments:
1. Greatly increase funding to national and subnational governments and private businesses, especially in LICs and LMICs, to carry out needed SDG investments.

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "The grim reality is that at the midpoint of the 2030 Agenda, the SDGs are far off track. At the global level, averaging across countries, not a single SDG is currently projected to be met by 2030, with the poorest countries struggling the most."
  ]
}
Extracted Arguments:
1. The grim reality is that at the midpoint of the 2030 Agenda, the SDGs are far off track. At the global level, averaging across countries, not a single SDG is 

## SGD 10: Inequality

In [12]:
topic = "SGD 10 (Inequality): Reduce inequality within and among countries"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "10"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "Despite this alarming development, the SDGs are still achievable. None of their objectives are beyond our reach."
  ]
}
Extracted Arguments:
1. Despite this alarming development, the SDGs are still achievable. None of their objectives are beyond our reach.

--- Processing Page 8 ---
Model Output: {
  "arguments": [
    "Further investment is needed in statistical capacity and data literacy to support long-term pathways for key SDG transformations."
  ]
}
Extracted Arguments:
1. Further investment is needed in statistical capacity and data literacy to support long-term pathways for key SDG transformations.

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "SDSN offers the following recommendations to accelerate progress over the remaining seven years to 2030, and to set even more ambitious targets to be achieved by 2050 under the SDG framework."
  ]
}
Extracted Arguments:
1. SDSN offers the following recomme

## SGD 11: Sustainable cities

In [13]:
topic = "SGD 11 (Sustainable Cities, Sustainable Communities): Make cities and human settlements inclusive, safe, resilient and sustainable"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "11"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "Despite this alarming development, the SDGs are still achievable. None of their objectives are beyond our reach."
  ]
}
Extracted Arguments:
1. Despite this alarming development, the SDGs are still achievable. None of their objectives are beyond our reach.

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "SDSN offers the following recommendations to accelerate progress over the remaining seven years to 2030, and to set even more ambitious targets to be achieved by 2050 under the SDG framework."
  ]
}

Note: The provided text does not contain any direct arguments supporting or opposing SGD 11 (Sustainable Cities, Sustainable Communities). Therefore, no JSON object is returned as there are no relevant arguments extracted.
Extracted Arguments:
1. SDSN offers the following recommendations to accelerate progress over the remaini

## SGD 12: Responsible Consumption, Responsible Production

In [14]:
topic = "SGD 12 (Responsible Consumption, Responsible Production): Ensure sustainable consumption and production patterns"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "12"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "In most high-income countries (HICs), automatic stabilizers, emergency expenditure, and recovery plans mitiged the impacts of these multiple crises on socioeconomic outcomes. Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land), even in countries that are largely to blame for the climate and biodiversity crises."
  ]
}
Extracted Arguments:
1. In most high-income countries (HICs), automatic stabilizers, emergency expenditure, and recovery plans mitiged the impacts of these multiple crises on socioeconomic outcomes. Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land), even in countries that are larg

## SGD 13: Climate change

In [15]:
topic = "SGD 13 (Climate change): Take urgent action to combat climate change and its impacts"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "13"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)"
  ]
}
Extracted Arguments:
1. only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many developing economies. As called for by United Nations Secretary-General António Guterres,2"
  ]
}
Extracted Arguments:
1. UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many deve

## SGD 14: Life bellow water

In [16]:
topic = "SGD 14 (Life bellow Water): Conserve and sustainably use the oceans, seas and marine resources for sustainable development"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "14"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)"
  ]
}
Extracted Arguments:
1. Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many developing economies."
  ]
}
Extracted Arguments:
1. UN Member States should adopt an SDG Stimulus, to close the massive financing gap faced by many developing economies.

--- Processing Page 14 ---
Model Output: {
  "argu

## SGD 15: Life on land

In [17]:
topic = "SGD 15 (Life on land): Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, and halt and reverse land degradation and halt biodiversity loss"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "15"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
    "arguments": [
        "Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)"
    ]
}
Extracted Arguments:
1. Only limited progress is being made on the environmental and biodiversity goals, including SDG 12 (Responsible Consumption and Production), SDG 13 (Climate Action), SDG 14 (Life Below Water), and SDG 15 (Life on Land)

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "SDSN recommends that UN Member States adopt an SDG Stimulus to close the massive financing gap faced by many developing economies, with five main components:"
  ]
}
Extracted Arguments:
1. SDSN recommends that UN Member States adopt an SDG Stimulus to close the massive financing gap faced by many developing e

## SGD 16: Peace, Justice, Strong Institutions

In [18]:
topic = "SGD 16 (Peace, Justice, Strong Institutions): Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "16"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "promote peaceful and inclusive societies for sustainable development"
  ]
}
Extracted Arguments:
1. promote peaceful and inclusive societies for sustainable development

--- Processing Page 8 ---
Model Output: {
  "arguments": []
}
Extracted Arguments:

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "As the world’s nations prepare to meet in September to review the progress the world has made so far towards achieving the SDGs, at the midpoint of the 2030 Agenda, SDSN emphasizes six areas for immediate action."
  ]
}
Extracted Arguments:
1. As the world’s nations prepare to meet in September to review the progress the world has made so far towards achieving the SDGs, at the midpoint of the 2030 Agenda, SDSN emphasizes six areas for immediate action.

--- Processing Page 14 ---
Model Output: {
  "arguments": [
    "All UN Member States should adopt long-term sustainable development pathways that provide a s

## SGD 17: Partnerships, sustainable development

In [19]:
topic = "SGD 17 (Partnerships, sustainable development):Strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development"
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "17"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development"
  ]
}
Extracted Arguments:
1. strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development

--- Processing Page 8 ---
Model Output: {
  "arguments": [
    "Further investment is needed in statistical capacity and data literacy to support long-term pathways for key SDG transformations."
  ]
}

Note: There were no verbatim arguments directly supporting or opposing "SGD 17 (Partnerships, sustainable development): Strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development" in the provided text.
Extracted Arguments:
1. Further investment is needed in statistical capacity and data literacy to support long-term pathways for key SDG transformations.

--- Processing Page 13 ---
Model Output: {
  "arguments": [
    "UN Member States should

## SGD 0: Overarching terms

In [20]:
topic = "SGD Overarching terms: Sustainable Development Goal, SDG, Agenda 2030, leave no one behind, Voluntary National Review, SDG transformations, "
prefix = 'GLOBAL_SGD2023_'
output_dir = "..\\Data\\Extracted Arguments No Keywords (all text)\\"
sgd_number = "0"

resultado = process_directory(input_dir = process_text_path, 
                  output_dir = output_dir, 
                  prefix = prefix,
                  model_name = model_name,
                  topic = topic,
                  sgd_number =sgd_number)

merged_output_path = os.path.join(output_dir, f"{prefix}_ArgsSGD{sgd_number}_{model_name.replace(':', '-')}.json")
save_to_json(resultado, merged_output_path)


--- Processing Page 7 ---
Model Output: {
  "arguments": [
    "At their core, the SDGs are an investment agenda: it is critical that UN Member States adopt and implement the SDG Stimulus and support a comprehensive reform of the global financial architecture."
  ]
}
Extracted Arguments:
1. At their core, the SDGs are an investment agenda: it is critical that UN Member States adopt and implement the SDG Stimulus and support a comprehensive reform of the global financial architecture.

--- Processing Page 8 ---
Model Output: {
  "arguments": [
    "The SDSN and its global network will double-down on efforts to implement the SDGs by 2030 and beyond."
  ]
}

Note: There were no sentences or phrases that clearly supported or argued for or against the topic of Sustainable Development Goal (SDG) Overarching terms, Agenda 2030, leave no one behind, Voluntary National Review, and SDG transformations within the provided text.
Extracted Arguments:
1. The SDSN and its global network will double-